In [1]:
import os
import gym
import torch
import pprint
import argparse
import numpy as np
from torch.utils.tensorboard import SummaryWriter

from tianshou.policy import DDPGPolicy
from tianshou.env import DummyVectorEnv
from tianshou.utils.net.common import Net
from tianshou.trainer import offpolicy_trainer
from tianshou.exploration import GaussianNoise
from tianshou.data import Collector, ReplayBuffer
from tianshou.utils.net.continuous import Actor, Critic

In [2]:
args = {
    "action_shape": (1, ),
    "actor_lr": 0.001,
    "batch_size": 128,
    "buffer_size": 20000,
    "collect_per_step": 4,
    "critic_lr": 0.001,
    "device": 'cpu',
    "epoch": 20, 
    "exploration_noise": 0.1,
    "gamma": 0.99,
    "ignore_done": 1,
    "layer_num": 1,
    "logdir": 'log',
    "max_action": 2.0,
    "n_step": 1,
    "render": 0.0,
    "rew_norm": 1,
    "seed": 0,
    "state_shape": (3, ),
    "step_per_epoch": 2400,
    "task": "gym_aigis:aigis-v0",
    "tau": 0.005,
    "test_noise": 0.1,
    "test_num": 1,
    "training_num": 1
}

In [8]:
env = gym.make('Pendulum-v0')

In [9]:
env.action_space

Box(-2.0, 2.0, (1,), float32)

In [11]:
env.observation_space

Box(-8.0, 8.0, (3,), float32)

In [13]:
env.reset()

array([-0.99998678,  0.00514175,  0.93663172])

In [15]:
env.step(np.array([1]))

(array([-0.99878093, -0.04936256,  1.09048803]), -9.926052020968166, False, {})

In [6]:
torch.set_num_threads(1)  # we just need only one thread for NN
env = gym.make(args['task'])
print("LC://\t", env.spec.reward_threshold)
if args['task'] == 'Pendulum-v0':
    env.spec.reward_threshold = -250
args['state_shape'] = env.observation_space.shape or env.observation_space.n
args['action_shape'] = env.action_space.shape or env.action_space.n
args['max_action'] = env.action_space.high[0]
# you can also use tianshou.env.SubprocVectorEnv
# train_envs = gym.make(args.task)
train_envs = DummyVectorEnv(
    [lambda: gym.make(args['task']) for _ in range(args['training_num'])])
# test_envs = gym.make(args.task)
test_envs = DummyVectorEnv(
    [lambda: gym.make(args['task']) for _ in range(args['test_num'])])

# seed
np.random.seed(args['seed'])
torch.manual_seed(args['seed'])
train_envs.seed(args['seed'])
test_envs.seed(args['seed'])
# model
net = Net(args['layer_num'], args['state_shape'], device=args['device'])
actor = Actor(
    net, args['action_shape'],
    args['max_action'], args['device']
).to(args['device'])
actor_optim = torch.optim.Adam(actor.parameters(), lr=args['actor_lr'])
net = Net(args['layer_num'], args['state_shape'],
          args['action_shape'], concat=True, device=args['device'])
critic = Critic(net, args['device']).to(args['device'])
critic_optim = torch.optim.Adam(critic.parameters(), lr=args['critic_lr'])
policy = DDPGPolicy(
    actor, actor_optim, critic, critic_optim,
    action_range=[env.action_space.low[0], env.action_space.high[0]],
    tau=args['tau'], gamma=args['gamma'],
    exploration_noise=GaussianNoise(sigma=args['exploration_noise']),
    reward_normalization=args['rew_norm'],
    ignore_done=args['ignore_done'],
    estimation_step=args['n_step'])
# collector
train_collector = Collector(
    policy, train_envs, ReplayBuffer(args['buffer_size']))
test_collector = Collector(
    policy, test_envs, action_noise=GaussianNoise(sigma=args['test_noise']))
# log
log_path = os.path.join(args['logdir'], args['task'], 'ddpg')
writer = SummaryWriter(log_path)

def save_fn(policy):
    torch.save(policy.state_dict(), os.path.join(log_path, 'policy.pth'))

def stop_fn(mean_rewards):
    return mean_rewards >= env.spec.reward_threshold

# trainer
result = offpolicy_trainer(
    policy, train_collector, test_collector, args['epoch'],
    args['step_per_epoch'], args['collect_per_step'], args['test_num'],
    args['batch_size'], stop_fn=stop_fn, save_fn=save_fn, writer=writer)
assert stop_fn(result['best_reward'])
print("done.")

deploy_result:	Good
[deploy-cdt-up] Wait for cdt benchmark result ...
[deploy-cdt-up] Wait for cdt benchmark result ...
[deploy-cdt-up] Wait for cdt benchmark result ...
[deploy-cdt-up] Wait for cdt benchmark result ...


ConnectionError: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))

In [7]:
net

NameError: name 'net' is not defined

In [6]:
pprint.pprint(result)
# Let's watch its performance!
env = gym.make(args['task'])
policy.eval()
collector = Collector(policy, env)
result = collector.collect(n_episode=1, render=args['render'])
print(f'Final reward: {result["rew"]}, length: {result["len"]}')

{'best_result': '-227.20 ± 190.77',
 'best_reward': -227.20418108255635,
 'duration': '26.41s',
 'test_episode': 200.0,
 'test_speed': '12530.12 step/s',
 'test_step': 40000,
 'test_time': '3.19s',
 'train_episode': 80.0,
 'train_speed': '689.05 step/s',
 'train_step': 16000,
 'train_time/collector': '2.91s',
 'train_time/model': '20.31s'}
Final reward: -135.07230201301684, length: 200.0


In [7]:
args

{'action_shape': (1,),
 'actor_lr': 0.001,
 'batch_size': 128,
 'buffer_size': 20000,
 'collect_per_step': 4,
 'critic_lr': 0.001,
 'device': 'cpu',
 'epoch': 20,
 'exploration_noise': 0.1,
 'gamma': 0.99,
 'ignore_done': 1,
 'layer_num': 1,
 'logdir': 'log',
 'max_action': 2.0,
 'n_step': 1,
 'render': 0.0,
 'rew_norm': 1,
 'seed': 0,
 'state_shape': (3,),
 'step_per_epoch': 2400,
 'task': 'Pendulum-v0',
 'tau': 0.005,
 'test_noise': 0.1,
 'test_num': 100,
 'training_num': 8}